# Setup

## Import libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv

from ipywidgets import interact, widgets, fixed
from plotly.express import imshow as imshow_interactive

## Import core modules

In [ ]:
import sys
import pathlib

REPO_DIR = pathlib.Path(__name__).resolve().parent.parent
SRC_DIR = str(REPO_DIR / "src")
sys.path.append(SRC_DIR)

In [ ]:
from scanplot.setup_logger import logger

In [ ]:
from scanplot.utils.read_image import read_image_rgb
from scanplot.utils.drawing import (
    draw_image,
    draw_points_on_canvas,
    draw_points_on_image,
)
from scanplot.view.parameters_widget import init_points_num_slider, init_points_density_slider

In [ ]:
from scanplot.core.template_match import template_match
from scanplot.core.preprocess import replace_black_pixels
from scanplot.core.conv_map_operations import normalize_map
from scanplot.core.process_template import (
    template_tresholding,
    crop_image,
    reconstruct_template_mask,
)
from scanplot.core.hough_transform import generalized_hough_transform
from scanplot.core.parameters_search import (
    linear_parameter_transform,
    get_corr_map_maximums,
)
from scanplot.core.nms import apply_nms

# Algorithm Pipeline

In [ ]:
DATA_PATH = pathlib.Path("../datasets/")

In [ ]:
PLOT_NUMBER = 59
MARKER_NUMBER = 1

PLOT_PATH = DATA_PATH / "plot_images" / f"plot{PLOT_NUMBER}.png"
TEMPLATE_PATH = (
    DATA_PATH / "marker_images" / f"plot{PLOT_NUMBER}_marker{MARKER_NUMBER}.png"
)

In [ ]:
image = ScanPlotImage(PLOT_PATH)

template = ScanPlotTemplate(TEMPLATE_PATH)

image.draw()
template.draw()

In [ ]:
image.preprocess()  # replace_black_pixels
template.preprocess()

In [ ]:
mask = template.get_mask(treshold_value=200)

additonal_mask = template.get_additional_mask(mask)

In [ ]:
correlation_map = TemplateMatchClassic(
    image, template, method, mask, norm_result=True
)

correlation_map_additional = TemplateMatchClassic(
    image, template, method, mask_additonal, norm_result=True
)

correlation_map_hough = TemplateMatchHough(
    image, template, norm_result=True, crop_result=True
)

# correlation_map.normalize()
# correlation_map_additional.normalize()
# correlation_map_hough.normalize()

In [ ]:
correlation_map_final = weighted_sum(
    corr_map1=correlation_map,
    corr_map2=correlation_map_hough,
    w1=1,
    w2=0.6
)

## Algorithm parameters

The algorithm has 2 parameters:
- Points Number
- Points Density


The impact of these parameters on the final result is shown in the images below:

<!-- ![](../docs/images/parameter_search_1.png)

![](../docs/images/parameter_search_2.png) -->


In [ ]:
detector = Detector(image, template, convolution_map_final)

In [ ]:
# parameters for drawing detections (may be customized)
figure_size = 12
marker_size = 200
marker_color = "yellow"
marker_type = "*"

display(detector.interactive_widget())

In [ ]:
## Enter manually or get from widget
optimal_points_num = 56
optimal_points_density = 50

detector.set_parameters(
    points_num=optimal_points_num, 
    points_density=optimal_points_density
)

In [ ]:
# parameters for drawing detections (may be customized)
figure_size = 12
marker_size = 200
marker_color = "yellow"
marker_type = "*"

detector.draw_result()

In [ ]:
detector.save_image_with_detections("where/to/save/image.png")

In [ ]:
pixel_points = detector.get_points()

In [ ]:
## mapper + converter

In [ ]:
from scanplot.view.coords_mapper import CoordinatesMapper
from scanplot.core.coords_converter import CoordinatesConverter

In [ ]:
mapper = CoordinatesMapper(image)

In [ ]:
display(mapper.interactive_widget())

In [ ]:
converter = CoordinatesConverter()
converter.import_parameters_from_mapper(mapper)

In [ ]:
converter = CoordinatesConverter()

factual_x_points, factual_y_points = converter.from_pixels(x_pixel=23, y_pixel=865)